In [5]:
#load in the generated story
with open("output/gpt-4o-mini/generated_story_level_2.txt", "r") as file:
    generated_story = file.read()

print(generated_story)

### Rapunzel

Once there was a man and a woman who wanted a baby. They wished for a long time. One day, the woman saw a garden from her window. It was full of beautiful flowers and herbs, but it had a high wall. The garden belonged to a powerful enchantress, and no one dared to go in.

The woman saw a plant called rampion in the garden. It looked so fresh and green that she really wanted to eat it. She felt so sick without it. Her husband noticed and asked, "What is wrong, dear wife?" 

"I want some rampion from the garden," she replied. "If I can't eat it, I will feel very sad!"

The husband cared for her and thought, "I will bring her some rampion." That night, he climbed over the wall and picked some. He gave it to his wife, and she made a salad. It tasted so good that she wanted more the next day.

The husband went back to the garden to get more rampion, but this time he saw the enchantress. She was angry. "How dare you steal my rampion?" she yelled. The husband was scared but expl

In [51]:
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, ValidationError
import json

load_dotenv()

class CharacterImagePrompt(BaseModel):
    character_name: str
    character_pose: str
    character_clothing_type: str
    character_clothing_style: str
    character_clothing_colour: str
    character_clothing_pattern: str
    character_clothing_texture: str
    character_expression: str
    character_prompt: str

class StoryImagePrompts(BaseModel):
    background_image_prompt: str
    character_image_prompts: list[CharacterImagePrompt]

class Scene(BaseModel):
    scene_no: int
    story_text: str
    story_image_prompts: StoryImagePrompts

class CharacterPermanentAttributes(BaseModel):
    name: str
    facial_features: str
    hair_colour: str
    hair_style: str
    body_type: str
    height: str
    weight: str

class StoryScenes(BaseModel):
    illustration_style: str
    characters: list[CharacterPermanentAttributes]
    scenes: list[Scene]

def split_story_into_scenes(story_text: str) -> StoryScenes:
    # Define a prompt to instruct the model to split the story into scenes
    prompt = f"Split the following story into scenes. Separate each scene with a new line. This is the story: {story_text}"

    # Call the OpenAI API
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

    completion = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": f"""You are an expert story illustrator creating a web app for children ages between 6-8 learning to read through stories. You are given a story text and before illustrating it, you need to 
                1. Read through the story text and chunk up the story into scenes. MAKE sure to have all the key scenes including the introduction, development, conflict, climax and resolution.
                2. Choose an illustration style for the story suitable for children ages between 6-8. Describe the style in detail so that can be used to generate consistent background images and characters. Describe the time period of the story, for example, for example, medieval, modern, futuristic, etc.
                3. Make a list of all the characters that appear in the story. If the same character is referred to with different ways, eg. man, husband, Mike, etc. you should refer to this character using his actual name if it is mentioned in the story, else refer to this character using the name that is most commonly used in the story and which is easily understood by someone reading your output. ALWAYS maintain consistency when referring to the same character throughout your output.
                3. For each scene, describe the background scene in detail, including the
                    - setting, 
                    - time of day
                    - weather
                    - the mood.
                    DO NOT INCLUDE any characters in the background description.
                4. For each scene, describe the characters in detail including their 
                    - clothing (include the type of clothes, composition, style, colour, texture, pattern, etc.)
                    - poses
                    - facial expressions 
                    so that it can be used to generate consistent images. The clothing description should be consistent for the same character unless it makes sense to change the character's clothing. Clothing description should include the type of clothes, composition, style, colour, texture, pattern, etc.              
                DO NOT alter the story text in any way.
                MAKE SURE TO use ALL the text provided in the story text."""
            },
            {
                "role": "user",
                "content": f"This is the story: {story_text}"
            }
        ],
        response_format=StoryScenes # Use "json" as the response format
    )

    # Parse the response into the StoryScenes model

    scenes = completion.choices[0].message.parsed
    return scenes
    
# Example usage
# scenes = split_story_into_scenes("Your story text here")
# print(scenes)

In [52]:
scenes = split_story_into_scenes(generated_story)
print(scenes)

illustration_style='The illustration style is colorful and whimsical, with a cartoon-like appearance featuring soft lines and exaggerated facial expressions to convey emotions. Characters are designed with large, expressive eyes and rounded features to appeal to children ages 6-8. The backgrounds are richly detailed with vibrant colors, showcasing magical elements like flowers and fantastical landscapes. The time period of the story is medieval, with elements like castles, towers, and lush forests that reflect a fairytale atmosphere, creating a sense of wonder and enchantment.' characters=[CharacterPermanentAttributes(name='The Woman', facial_features='Soft, gentle facial features', hair_colour='Brown', hair_style='Long, flowing', body_type='Average', height='Medium', weight='Light'), CharacterPermanentAttributes(name='The Man', facial_features='Strong jawline, kind eyes', hair_colour='Dark brown', hair_style='Short, slightly messy', body_type='Tall', height='Tall', weight='Athletic'),

In [53]:
scenes_json = scenes.model_dump(mode="json")
print(json.dumps(scenes_json, indent=4))

{
    "illustration_style": "The illustration style is colorful and whimsical, with a cartoon-like appearance featuring soft lines and exaggerated facial expressions to convey emotions. Characters are designed with large, expressive eyes and rounded features to appeal to children ages 6-8. The backgrounds are richly detailed with vibrant colors, showcasing magical elements like flowers and fantastical landscapes. The time period of the story is medieval, with elements like castles, towers, and lush forests that reflect a fairytale atmosphere, creating a sense of wonder and enchantment.",
    "characters": [
        {
            "name": "The Woman",
            "facial_features": "Soft, gentle facial features",
            "hair_colour": "Brown",
            "hair_style": "Long, flowing",
            "body_type": "Average",
            "height": "Medium",
            "weight": "Light"
        },
        {
            "name": "The Man",
            "facial_features": "Strong jawline, ki

In [54]:
#write the output to file
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "w") as file:
    json.dump(scenes_json, file, indent=4)

In [55]:
#read the file storylabs/backend/output/gpt-4o-mini/generated_story_scenes_level_2.json
import json
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "r") as file:
    scenes_json = json.load(file)




## Flux

In [10]:
!poetry add replicate

Using version ^1.0.4 for replicate

Updating dependencies
Resolving dependencies... (1.1s)Resolving dependencies... (1.0s)

Package operations: 1 install, 0 updates, 0 removals

  - Installing replicate (1.0.4): Pending...
  - Installing replicate (1.0.4): Downloading... 0%
  - Installing replicate (1.0.4): Downloading... 100%
  - Installing replicate (1.0.4): Installing...
  - Installing replicate (1.0.4)

Writing lock file


In [13]:
import replicate
load_dotenv()
REPLICATE_API_TOKEN = os.environ["REPLICATE_API_TOKEN"]

r8_TEoR0ltdPpRx2r6VZEwfSXmvI5kDfgp4dFS6M


In [56]:
#load the image prompts from file
with open("output/gpt-4o-mini/generated_story_scenes_level_2.json", "r") as file:
    story_image_prompts = json.load(file)

In [57]:
character_name='Rapunzel'

In [58]:
#get all the clothing for character Woman
character_image_prompts = []
for scene in story_image_prompts["scenes"]:
    if scene['story_image_prompts'].get('character_image_prompts'):
        for character in scene['story_image_prompts']['character_image_prompts']:
            if character.get('character_name', '') == character_name:
                character_image_prompts.append(character)

print(character_image_prompts)

[{'character_name': 'Rapunzel', 'character_pose': 'Standing at the window, looking out longingly.', 'character_clothing': 'A long flowing gown with bright colors, decorated with flowers; her hair cascades down.', 'character_expression': 'Curious and a bit sad.', 'character_prompt': 'Rapunzel gazing down from her tower.'}, {'character_name': 'Rapunzel', 'character_pose': 'Leaning out the window, holding her hair.', 'character_clothing': 'A long flowing gown with bright colors, decorated with flowers.', 'character_expression': 'Eager and hopeful.', 'character_prompt': 'Rapunzel ready to let down her hair.'}, {'character_name': 'Rapunzel', 'character_pose': 'Looking at the prince with wide eyes, appearing nervous but intrigued.', 'character_clothing': 'A long flowing gown with bright colors and floral decorations.', 'character_expression': 'Surprised and curious.', 'character_prompt': 'Rapunzel meeting the prince for the first time.'}, {'character_name': 'Rapunzel', 'character_pose': 'Hol

In [59]:
character_permanent_attributes = next((character for character in story_image_prompts["characters"] if character["name"] == 'Rapunzel'), None)
print(character_permanent_attributes)


{'name': 'Rapunzel', 'facial_features': 'Large, bright eyes', 'hair_colour': 'Golden', 'hair_style': 'Long, flowing', 'body_type': 'Slim', 'height': 'Tall', 'weight': 'Light'}


In [60]:
image_prompt_text = f"""{character_permanent_attributes} 
{character_image_prompts[0]}
Illustration style: {story_image_prompts['illustration_style']}
"""

#remove name from the image prompt text
image_prompt_text = image_prompt_text.replace(character_name, '')
print(image_prompt_text)

{'name': '', 'facial_features': 'Large, bright eyes', 'hair_colour': 'Golden', 'hair_style': 'Long, flowing', 'body_type': 'Slim', 'height': 'Tall', 'weight': 'Light'} 
{'character_name': '', 'character_pose': 'Standing at the window, looking out longingly.', 'character_clothing': 'A long flowing gown with bright colors, decorated with flowers; her hair cascades down.', 'character_expression': 'Curious and a bit sad.', 'character_prompt': ' gazing down from her tower.'}
Illustration style: The illustration style is colorful and whimsical, with a cartoon-like appearance featuring soft lines and exaggerated facial expressions to convey emotions. Characters are designed with large, expressive eyes and rounded features to appeal to children ages 6-8. The backgrounds are richly detailed with vibrant colors, showcasing magical elements like flowers and fantastical landscapes. The time period of the story is medieval, with elements like castles, towers, and lush forests that reflect a fairyta

In [48]:
input = {
    "prompt": image_prompt_text,
    "prompt_upsampling": False, 
    "aspect_ratio": "16:9", 
    "output_format": "png", 
    "seed": 123456
}

output = replicate.run(
    "black-forest-labs/flux-1.1-pro",
    input=input
)


In [49]:

#make output directory if it doesn't exist
os.makedirs("output/images", exist_ok=True)
with open(f"output/images/{character_name}.png", "wb") as file:
    file.write(output.read())